In [7]:
using ITensors
using KrylovKit
include("./wq_lib/newabstractprojmpo.jl")
#include("new_functions.jl")
function QNITensor(qnumber,link1,link2)
    D = ITensor(link1,link2)
    for n1 = 1:nblocks(link1)
        for n2 = 1:nblocks(link2)
            if qn(link1,n1)+qn(link2,n2)==qnumber
                a1 =1
                a2 = 1
                for i = 1:n1
                    if i != 1
                        a1 += blockdim(link1,i-1)
                    end
                end
                for i = 1:n2
                    if i != 1
                        a2 += blockdim(link2,i)
                    end
                end
                D[a1,a2] = 1
            end
        end
    end
    return D
end
function swapUnitCells(A)
    N = length(A)
    if typeof(A)==MPS
        B = MPS(N)
    end
    if typeof(A)==MPO
        B = MPO(N)
    end
    for i =1:(N÷2)
        B[i] = A[N÷2+i]
        B[N÷2+i]=A[i]
    end
    return B
end
let
    #define total Hamiltonian as H, the total wavefunction as psi
    N0 = 16
    N = 100
    sites = siteinds("S=1/2",N;conserve_qns=true)
    os = OpSum()
    for j in 1:(N-1)
        os .+= 0.5,"S+",j,"S-",j+1
        os .+= 0.5,"S-",j,"S+",j+1
        os .+= "Sz",j,"Sz",j+1
    end
    H = MPO(os,sites)
    psi = MPS(sites)
    #define initial Hamiltonian as H0, the initial wavefunction as psi0 
    sites0 = siteinds("S=1/2",N0;conserve_qns=true)
    state = [isodd(n) ? "Up" : "Dn" for n in 1:N0]
    psi0 = MPS(sites0,state)
    H0 = MPO(N0)
    for i=1:(N0÷2)
        H0[i] = deepcopy(H[i])
        H0[N0+1-i] = deepcopy(H[N+1-i])
        # we only modify the site index of psi and psi0 to be the same
        # we do it to make the same MPO can be applied on both psi and psi0
        replaceind!(psi0[i],siteindex(psi0,i;tags="Site"),siteindex(psi,i;tags="Site"))
        replaceind!(psi0[N0+1-i],siteindex(psi0,N0+1-i;tags="Site"),siteindex(psi,N+1-i;tags="Site"))
    end
    link1 = uniqueinds(H0[N0÷2],H0[N0÷2-1];tags="Link")
    link2 = uniqueinds(H0[N0÷2+1],H0[N0÷2+2],tags="Link")
    replaceinds!(H0[N0÷2],link1,link2)
    #set DMRG parameters and do DMRG calculation, we get the groud state properties of the initial system
    nsweeps = 8
    maxdim = 10
    cutoff = 1e-10
    energy0,psi0 = dmrg(H0,psi0; nsweeps, maxdim, cutoff)
    TotalQN = totalqn(psi0)
    println("initial dmrg finished! Total Qutumnumber = $TotalQN")
    # Initial System and Environment
    unit_cell = 4
    psi0 = ITensors.orthogonalize!(psi0,N0÷2)
    bond0 = psi0[N0÷2]*psi0[N0÷2+1]
    ind0 = inds(bond0)
    U,S,V = svd(bond0,(ind0[1],ind0[2]))
    psi0[N0÷2] = U
    psi0[N0÷2+1] = V
    S ./= norm(S)
    HL = psi0[1]*H[1]*dag(prime(psi0[1]))
    HR = psi0[N0]*H[N]*dag(prime(psi0[N0]))
    psi[1] = psi0[1]
    psi[N] = psi0[N0]
    for i = 2:(N0÷2)
        HL = HL*psi0[i]
        HL = HL*H[i]
        HL = HL*dag(prime(psi0[i]))
        HR = HR*psi0[N0+1-i]
        HR = HR*H[N+1-i]
        HR = HR*dag(prime(psi0[N0+1-i]))
        psi[i] = psi0[i]
        psi[N+1-i] = psi0[N0+1-i]
    end
    # Initial Cell
    psi_cell = MPS(unit_cell)
    H_cell = MPO(unit_cell)
    for i = 1:unit_cell
        psi_cell[i] = psi0[N0÷2-unit_cell÷2+i]
    end
    println("initial cell finished!")
    #IDMRG Grow
    for L = N0:unit_cell:(N-unit_cell)
        psi_cell = swapUnitCells(psi_cell)
        psi_cell[1] = psi_cell[1]*S
        psi_cell[unit_cell] = psi_cell[unit_cell]*S
        for i = 1:unit_cell÷2
            H_cell[i] = deepcopy(H[L÷2+i])
            H_cell[unit_cell+1-i]=deepcopy(H[N+1-L÷2-i])
            replaceind!(psi_cell[i],findindex(psi_cell[i];tags="Site"),sites[L÷2+i])
            replaceind!(psi_cell[unit_cell+1-i],findindex(psi_cell[unit_cell+1-i];tags="Site"),sites[N+1-L÷2-i])
        end
        if unit_cell >2
            l_link = uniqueinds(H_cell[unit_cell÷2],H_cell[unit_cell÷2-1];tags="Link")
            r_link = uniqueinds(H_cell[unit_cell÷2+1],H_cell[unit_cell÷2+2],tags="Link")
        else
            l_link = [inds(H_cell[1];tags="Link")[end]]
            r_link = [inds(H_cell[2];tags="Link")[1]]
        end
        replaceinds!(H_cell[unit_cell÷2],l_link,r_link)
        # control QNs
        target_QN = QN(("Sz",0))
        if unit_cell>2
            link1 = dag(uniqueinds(psi_cell[unit_cell÷2],psi_cell[unit_cell÷2-1];tags="Link"))
            link2 = dag(uniqueinds(psi_cell[unit_cell÷2+1],psi_cell[unit_cell÷2+2];tags="Link"))
        else
            link1 = [dag(inds(psi_cell[1])[end])]
            link2 = [dag(inds(psi_cell[2])[1])]
            print(link1)
            print(link2)
            print(TotalQN+TotalQN-target_QN)
        end
        #D = QNITensor(TotalQN+TotalQN-target_QN,link1[1],link2[1])
        D = randomITensor(TotalQN+TotalQN-target_QN,link1[1],link2[1])
        @assert norm(D)>0 "cannot get non-zero D Tensor"
        psi_cell[unit_cell÷2] = psi_cell[unit_cell÷2]*D
        # do dmrg: optimize psi_cell
        PH_cell = NewProjMPO(H_cell,HL,HR)
        if !isortho(psi_cell) || orthoCenter(psi_cell)!=1
            psi_cell=ITensors.orthogonalize!(PH_cell,psi_cell,1) # set the orthogonal center at the first site
        end
        PH_cell = position!(PH_cell,psi_cell,1) #Effective Hamiltonian for the first-two sites
        energy = 0.0
        for sw in 1:nsweeps
            for (b,ha) in sweepnext(unit_cell)
                PH_cell = position!(PH_cell,psi_cell,b)
                phi = psi_cell[b]*psi_cell[b+1]
                vals,vecs = eigsolve(PH_cell,phi,1,:SR;ishermitian=true)
                energy = vals[1]
                phi = vecs[1]
                ortho = ha==1 ? "left" : "right" #judge the sweep direction
                spec = replacebond!(PH_cell,psi_cell,b,phi;maxdim=maxdim,ortho=ortho,normalize=true)
            end
        end
        println("length is $(L+unit_cell), energy = $energy, TotalQN = $target_QN")
        
        # prepare next psi_cell
        psi_cell=ITensors.orthogonalize!(psi_cell,unit_cell÷2)
        bond_cell = psi_cell[unit_cell÷2]*psi_cell[unit_cell÷2+1]
        ind_cell = inds(bond_cell)
        U,S,V = svd(bond_cell,(ind_cell[1],ind_cell[2]))
        psi_cell[unit_cell÷2] = U
        psi_cell[unit_cell÷2+1] = V
        S ./= norm(S)
        for i = 1:unit_cell÷2
            psi[L÷2+i] = psi_cell[i]
            psi[N-L÷2+1-i] = psi_cell[unit_cell+1-i]
            
            HL = HL*psi_cell[i]
            HL = HL*H[L÷2+i]
            HL = HL*dag(prime(psi_cell[i]))
                    
            HR = HR*psi_cell[unit_cell+1-i]
            HR = HR*H[N+1-L÷2-i]
            HR = HR*dag(prime(psi_cell[unit_cell+1-i]))
        end
        TotalQN = target_QN
        println("length $(L+unit_cell) finished TotalQN = $(TotalQN)")
    end
    psi[N÷2] = psi[N÷2]*S
    println("end of iDMRG grow!")
    
end
            

After sweep 1 energy=-6.846209263896596  maxlinkdim=4 maxerr=8.40E-16 time=0.047
After sweep 2 energy=-6.911486484591256  maxlinkdim=10 maxerr=4.55E-06 time=0.087
After sweep 3 energy=-6.9116579478920155  maxlinkdim=10 maxerr=5.81E-06 time=0.098
After sweep 4 energy=-6.911658158778879  maxlinkdim=10 maxerr=5.65E-06 time=0.080
After sweep 5 energy=-6.911658158386447  maxlinkdim=10 maxerr=5.60E-06 time=0.092
After sweep 6 energy=-6.911658158373182  maxlinkdim=10 maxerr=5.60E-06 time=0.072
After sweep 7 energy=-6.911658158372975  maxlinkdim=10 maxerr=5.60E-06 time=0.055
After sweep 8 energy=-6.911658158372983  maxlinkdim=10 maxerr=5.60E-06 time=0.064
initial dmrg finished! Total Qutumnumber = QN("Sz",0)
initial cell finished!
length is 20, energy = -8.68232752487482, TotalQN = QN("Sz",0)
length 20 finished TotalQN = QN("Sz",0)
length is 24, energy = -10.453530225512282, TotalQN = QN("Sz",0)
length 24 finished TotalQN = QN("Sz",0)
length is 28, energy = -12.225033193737769, TotalQN = QN("S